In [12]:
import pyodbc
import pandas as pd

In [13]:
def connect_to_db(driver, server, database, username, password):
    """
    String para conectar a la base de datos SQL y Sypnase Analytics.
    """
    conn_str = (
        f"Driver={{{driver}}};"
        f"Server={server};"
        f"Database={database};"
        f"Uid={username};"
        f"Pwd={password};"
        f"Encrypt=yes;"
        f"TrustServerCertificate=no;"
        f"Connection Timeout=30;"
    )
    return pyodbc.connect(conn_str)

In [14]:
def extract_data(conn, query):
    """
    Extraer los datos de la base de datos SQL - Gestion de Clientes Seguros XYZ
    """
    return pd.read_sql(query, conn)

In [15]:
def validate_phone(phone):
    """
    Validar que el número teléfonico sea valido
    """
    if phone.isdigit() and len(phone) == 10:
        return phone
    else:
        return "DatoIncorrecto"

In [16]:
def detect_inconsistencies(df):
    """
    Detectar Inconsistencias en la Data.
    """
    inconsistencies = {
        'null_values': {},
        'invalid_phone': [],
        'negative_income': [],
        'duplicates': []
    }

    # Encontrar valores NULL
    inconsistencies['null_values'] = df.isnull().sum()

    # Validar numeros telefonicos
    invalid_phone_rows = df[df['Telefono'] == 'DatoIncorrecto']
    inconsistencies['invalid_phone'] = invalid_phone_rows.index.tolist()

    # Encontrar Ingresos Anuales negativos
    negative_income_rows = df[df['IngresoAnual'] < 0]
    inconsistencies['negative_income'] = negative_income_rows.index.tolist()

    # Encontrar datos duplicados basados en ClienteID
    duplicates = df[df.duplicated(subset='ClienteID', keep=False)]
    inconsistencies['duplicates'] = duplicates.index.tolist()

    return inconsistencies


In [21]:
def main():
    # Datos de conexión a la base de datos del sistema de gestión de clientes
    dw_driver = 'ODBC Driver 18 for SQL Server'
    dw_server = 'tcp:workspacesegurosxyz.sql.azuresynapse.net,1433'
    dw_database = 'segurosxyz_sqlpool'
    dw_username = 'sqladminuser'
    dw_password = 'Segurosxyz.2024'

    # Conectar a la base de datos del sistema de gestión de clientes
    conn_clientes = connect_to_db(dw_driver, dw_server, dw_database, dw_username, dw_password)
    query_clientes = "SELECT * FROM mart.DimClientes"
    df_clientes = extract_data(conn_clientes, query_clientes)
    
    # Detectar inconsistencias
    inconsistencies = detect_inconsistencies(df_clientes)

    # Mostrar las inconsistencias
    print("Inconsistencias encontradas:")
    print(f"Valores nulos:\n{inconsistencies['null_values']}\n")
    print(f"Filas con teléfonos marcados como Dato Incorrecto: {len(inconsistencies['invalid_phone'])}\n")
    print(f"Filas con ingresos anuales negativos: {len(inconsistencies['negative_income'])}\n")
    print(f"Filas duplicadas basadas en ClienteID: {len(inconsistencies['duplicates'])}\n")

In [23]:
if __name__ == "__main__":
    main()

C:\Users\CATALINA\AppData\Local\Temp\ipykernel_22848\333626351.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


Inconsistencias encontradas:
Valores nulos:
ClienteID          0
NombreCompleto     0
Direccion          0
Telefono           0
FechaNacimiento    0
Sexo               0
EstadoCivil        0
Profesion          0
IngresoAnual       0
FechaRegistro      0
Estado             0
dtype: int64

Filas con teléfonos marcados como Dato Incorrecto: 29

Filas con ingresos anuales negativos: 0

Filas duplicadas basadas en ClienteID: 0

